In [1]:
from PIL import Image
from glob import glob
import os 
import numpy as np
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Dense, Flatten,GlobalAveragePooling2D,Dropout
from sklearn.model_selection import train_test_split
from keras.applications.mobilenet_v2 import MobileNetV2
import matplotlib.pyplot as plt 
import pickle

In [2]:
# Get a list of all the image file paths in a directory
data = []
labels = []
root = "data/"
categories = os.listdir(root)
for c in range(len(categories)):
    category = categories[c]
    path = os.path.join(root,category)
    image_files = glob(path+'/*.jpg')
    # Loop through the file paths and open each image
    for file in image_files:
        image = Image.open(file).convert('RGB')
        # Resize the image using the Image.resize() method
        image = image.resize((224, 224))
        data.append(np.array(image)/255)
        labels.append(c)

In [6]:
X = np.array(data)
y = np.array(labels)

In [3]:
categories

['cloudy', 'desert', 'green_area', 'water']

In [4]:
with open('categories.pickle','wb') as f : 
    pickle.dump(categories,f)

In [7]:
n = len(set(y))

In [8]:
X_train , X_test , y_train ,y_test = train_test_split(X,y,random_state=42)

In [9]:
base_model = MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
                              base_model,
                              GlobalAveragePooling2D(),
                              Dense(1024, activation = 'relu'),
                              Dropout(rate = 0.2),
                              Dense(n, activation='softmax')
                            ])

model.compile(optimizer = "adam",
                loss = "sparse_categorical_crossentropy",
                metrics=["accuracy"])

In [10]:
history = model.fit(X_train,y_train,epochs=1,validation_data=(X_test,y_test))

132/132 [==============================] - 87s 643ms/step - loss: 0.1792 - accuracy: 0.9391 - val_loss: 0.0528 - val_accuracy: 0.9844


In [11]:
model.save("model.h5")